# Making Better Models

*This worksheet was originally designed by [Erin George](https://www.math.ucla.edu/~egeo/) (Department of Mathematics, UCLA). It has been subsequently revised by later TAs and instructors.*

Before, I've supplied data and a model, and let you all mess with the model to try to improve.  This is great practice!  This time, let's start with a model and try to improve it together.  Hopefully you can see some new techniques this way.

Let's use the [Fortune 1000](https://www.kaggle.com/winston56/fortune-500-data-2021) dataset from Kaggle.  I've put it on CCLE as well.  When using real-world datasets like this one, we aren't going to get perfect models.  This is just a reality with machine learning.  This is especially true with datasets that capture complex phenonmena, such as social science or economics datasets.  Still, there's no harm in trying.  Let's see how good we can make it!

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('Fortune_1000.csv')
display(df.head())
df.shape

This dataset needs a lot of cleaning and preprocessing.  Let's only do what is necessary to produce a working model at first.  Then we can see if there's anything else we need to do.

First, let's remove the columns that we wouldn't want to use as part of our models.  These are going to be the columns that contain very specific and not generalizeable information, such as name and CEO.

In [ ]:
df.drop(columns=['company', 'city', 'CEO', 'Website', 'Ticker'], inplace=True)
display(df.head())
df.shape

Next, when we read the information for the dataset, we see that a lot of the information is only available for the top 500 companies.  So we will either need to remove half of the data points or remove some of the features.  Which should we do?  Well, it depends on what we actually want to model.  A reasonable thing to model here would be the `rank_change` column as that sounds like something we'd actually want to predict (if we have an accurate model for this, we might be able to predict next year's rank change, for instance).  This is a feature that is only available for companies that are in the top 500 this year AND last year.  These are the companies that have 'no' for the `newcomer` column.  So let's take that.  Once we do this, the `newcomer` column will be pretty useless, so we can remove it.

In [ ]:
df = df[df['newcomer'] == 'no']
df.drop(columns=['newcomer'], inplace=True)
display(df.head())
df.shape

After this, we can remove the data with blanks.  It's just `Market Cap` that has missing data.  This seems like a useful column to keep, so let's remove the rows with missing data there.  Also, this column is actually stored as strings.  That's no good: let's make the values floats.  Same with `prev_rank`.

In [ ]:
df.dropna(inplace=True)
df = df[df['Market Cap'] != '-']
df['Market Cap'] = df['Market Cap'].astype(float)
df['prev_rank'] = df['prev_rank'].astype(float)

display(df.head())
df.shape

Lastly, let's remove the `rank_change` columns to get our labels.  We can deduce the rank change pretty easily from `rank` and `prev_rank`, so we should remove one of those as well.  If we're trying to predict the change in rank in the future, it makes more sense to use the `prev_rank`, so we'll keep that one.

In [ ]:
df.drop(columns=['rank'], inplace=True)
y = df['rank_change']
x = df.drop(columns=['rank_change'])

display(x.head())
display(y.head())

Next we make our test and validation sets.  We'll use the `random_state` parameter of `train_test_split` to ensure the test set doesn't change from run to run.

In [ ]:
from sklearn.model_selection import train_test_split

x_trv, x_test, y_trv, y_test = train_test_split(x, y, random_state = 15, test_size = 0.2)
x_train, x_val, y_train, y_val = train_test_split(x_trv, y_trv, test_size = 0.2)

One more thing with our data.  We have categorical features.  We should encode them!  We'll use the `OneHotEncoder` from scikit-learn since it's the option that makes the most sense for our data (the other ones are mean to used on the labels or on ordered classes).  We don't need to use this on the yes/no data, we can just set that to 0 or 1 ourselves.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

x_train.loc[:,'ceo_founder'] = (x_train['ceo_founder'] == 'yes').astype(int)
x_val.loc[:,'ceo_founder']   = (x_val['ceo_founder']   == 'yes').astype(int)
x_test.loc[:,'ceo_founder']  = (x_test['ceo_founder']  == 'yes').astype(int)

x_train.loc[:,'ceo_woman'] = (x_train['ceo_woman'] == 'yes').astype(int)
x_val.loc[:,'ceo_woman']   = (x_val['ceo_woman']   == 'yes').astype(int)
x_test.loc[:,'ceo_woman']  = (x_test['ceo_woman']  == 'yes').astype(int)

x_train.loc[:,'profitable'] = (x_train['profitable'] == 'yes').astype(int)
x_val.loc[:,'profitable']   = (x_val['profitable']   == 'yes').astype(int)
x_test.loc[:,'profitable']  = (x_test['profitable']  == 'yes').astype(int)

x_train.reset_index(inplace=True, drop=True)
x_val.reset_index(inplace=True, drop=True)
x_test.reset_index(inplace=True, drop=True)

ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(x_train[['state', 'sector']])
x_train = x_train.join(pd.DataFrame(ohe.transform(x_train[['state', 'sector']]).toarray()))
x_val   = x_val.join(pd.DataFrame(ohe.transform(x_val[['state', 'sector']]).toarray()))
x_test  = x_test.join(pd.DataFrame(ohe.transform(x_test[['state', 'sector']]).toarray()))

x_train.drop(columns=['sector', 'state'], inplace=True)
x_val.drop(columns=['sector', 'state'], inplace=True)
x_test.drop(columns=['sector', 'state'], inplace=True)

x_train.head()

Now we can make our model!  We want to predict the `rank_change`.  Is this regression or classification?  If you listen to what I said before, you would probably think it's classification.  Even though the change in rank is a numerical value, it's a discrete variable that only takes integer values.

Turns out, what I said then was a little wrong, sorry!

So here, even though a rank change of 4.5 would never actually occur, a rank change of "about 4.5" still makes a lot of sense.  Also, the changes are *ordered* while classification problems are *unordered*.  And, even though the rank changes can only be integer values, there's still an infintie number of values it can potentionally be.  In classification, we want to have a finite number of classes.

That being said, we could still frame this as classification if we wanted to.  But regression makes a little bit more sense.  Here's the model we will be starting with.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

model = keras.models.Sequential([
    keras.layers.Dense(units = 50, activation = 'sigmoid', input_shape = (x_train.shape[1],)),
    keras.layers.Dense(units = 10, activation = 'sigmoid'),
    keras.layers.Dense(units = 1),
])

model.summary()

Then we train:

In [ ]:
model.compile(optimizer='adam',
              loss='mse')
model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), verbose=0)

A good measure at the end to see how predict the model is to look at the $r^2$ between the predicted values and the actual values.

In [ ]:
train_mse = model.evaluate(x_train, y_train)
train_var = y_train.var()

val_mse = model.evaluate(x_val, y_val)
val_var = y_val.var()

print(f'Train MSE: {train_mse}')
print(f'Train r^2: {(train_var - train_mse)/train_var}')

print(f'Validation MSE: {val_mse}')
print(f'Validation r^2: {(val_var - val_mse)/val_var}')

This is... not good.  How can we fix it?

Well, let's rethink some of our features.  The "state" feature is a little suspicious.  We still have the dataset before we did any encoding.  Let's see exactly the distribution of states in the data.

In [ ]:
df.groupby('state').aggregate('count')['rank_change']

There's a lot of states that barely appear in the data.  Maybe we should reconsider things?  What could we do with this instead?

Well, one option is to group up states so that there are fewer classes and more data points per class.  Maybe every state with a few number of companies becomes an "other" class, or we group by region of the US instead of by state.  This is actually a pretty good idea, but would be a little tricky to do.

The other option is to just remove the column.  This actually isn't a bad as it sounds: a confusing data point isn't much better than no data point at all.  Let's do that.  There's not really an easy way to undo the encoding we just did, so let's just start from splitting again.  Let's also reorder some of the encoding so we don't have to keep doing it.

In [ ]:
x.drop(columns=['state'], inplace=True)
x.loc[:,'ceo_founder'] = (x['ceo_founder'] == 'yes').astype(int)
x.loc[:,'ceo_woman']   = (x['ceo_woman']   == 'yes').astype(int)
x.loc[:,'profitable']  = (x['profitable'] == 'yes').astype(int)

x_trv, x_test, y_trv, y_test = train_test_split(x, y, random_state = 15, test_size = 0.2)
x_train, x_val, y_train, y_val = train_test_split(x_trv, y_trv, test_size = 0.2)

x_train.reset_index(inplace=True, drop=True)
x_val.reset_index(inplace=True, drop=True)
x_test.reset_index(inplace=True, drop=True)

ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(x_train[['sector']])
x_train = x_train.join(pd.DataFrame(ohe.transform(x_train[['sector']]).toarray()))
x_val   = x_val.join(pd.DataFrame(ohe.transform(x_val[['sector']]).toarray()))
x_test  = x_test.join(pd.DataFrame(ohe.transform(x_test[['sector']]).toarray()))

x_train.drop(columns=['sector'], inplace=True)
x_val.drop(columns=['sector'], inplace=True)
x_test.drop(columns=['sector'], inplace=True)

x_train.head()

Does this make it better?  Let's find out!

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(units = 50, activation = 'sigmoid', input_shape = (x_train.shape[1],)),
    keras.layers.Dense(units = 10, activation = 'sigmoid'),
    keras.layers.Dense(units = 1),
])

model.compile(optimizer='adam',
              loss='mse')
model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), verbose=0)

train_mse = model.evaluate(x_train, y_train)
train_var = y_train.var()

val_mse = model.evaluate(x_val, y_val)
val_var = y_val.var()

print(f'Train MSE: {train_mse}')
print(f'Train r^2: {(train_var - train_mse)/train_var}')

print(f'Validation MSE: {val_mse}')
print(f'Validation r^2: {(val_var - val_mse)/val_var}')

It's still pretty bad, but it looks a little better.  What else could we fix?  We never messed with the quantitative variables, maybe we should modify them a little bit?

We might be tempted to standardize them.  What if we did?  Let's take a look at one of them: number of employees.

In [ ]:
import matplotlib.pyplot as plt

plt.hist(x['num. of employees'])
plt.show()

These take a LARGE range of values.  Even if we standardize them so that the mean is zero and the standard deviation is one, most of the data is going to be clumped together and there's going to be a few outliers.  Is there something better we can do?

Well, a lot of data that's spread out like this is actually pretty nicely distributed on a log scale.  Let's see how our data looks in a log scale!

In [ ]:
plt.hist(np.log(x['num. of employees']))
plt.show()

Oh, that's *much* nicer.  Does it work with the other ones?

In [ ]:
fig, axs = plt.subplots(1, 2)

axs[0].hist(x['revenue'])
axs[1].hist(np.log(x['revenue']))
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2)

axs[0].hist(x['Market Cap'])
axs[1].hist(np.log(x['Market Cap']))
plt.show()

These distributions aren't as good as the first one, but it's still much better under a log scale.

Profit is another thing we should check.  Unfortunately, the profit is sometimes negative, which doens't work for log.  What can we do?

Well, there's a "made a profit?" column, so we can take the absolute value of the profit without losing any information.  This does destroy the ordinal nature of the profit column though, so this doesn't seem ideal.

Another ideal is to combine profit and revenue to create an expenses column, which would be positive numbers.  Let's try that for now.

In [ ]:
fig, axs = plt.subplots(1, 2)

axs[0].hist(x['revenue'] - x['profit'])
axs[1].hist(np.log(x['revenue'] - x['profit']))
plt.show()

This looks pretty good too!   Let's make these changes.

In [ ]:
x['profit']            = np.log(x['revenue'] - x['profit'])
x['num. of employees'] = np.log(x['num. of employees'])
x['revenue']           = np.log(x['revenue'])
x['Market Cap']        = np.log(x['Market Cap'])
x.rename(columns = {'revenue': 'log(revenue)',
                    'profit': 'log(expenses)',
                    'num. of employees': 'log(employees)',
                    'Market Cap': 'log(market cap)'}, inplace=True)
x.head()

In [ ]:
x_trv, x_test, y_trv, y_test = train_test_split(x, y, random_state = 15, test_size = 0.2)
x_train, x_val, y_train, y_val = train_test_split(x_trv, y_trv, test_size = 0.2)

x_train.reset_index(inplace=True, drop=True)
x_val.reset_index(inplace=True, drop=True)
x_test.reset_index(inplace=True, drop=True)

ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(x_train[['sector']])
x_train = x_train.join(pd.DataFrame(ohe.transform(x_train[['sector']]).toarray()))
x_val   = x_val.join(pd.DataFrame(ohe.transform(x_val[['sector']]).toarray()))
x_test  = x_test.join(pd.DataFrame(ohe.transform(x_test[['sector']]).toarray()))

x_train.drop(columns=['sector'], inplace=True)
x_val.drop(columns=['sector'], inplace=True)
x_test.drop(columns=['sector'], inplace=True)

x_train.head()

Again, the model:

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(units = 50, activation = 'sigmoid', input_shape = (x_train.shape[1],)),
    keras.layers.Dense(units = 10, activation = 'sigmoid'),
    keras.layers.Dense(units = 1),
])

model.compile(optimizer='adam',
              loss='mse')
model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), verbose=0)

train_mse = model.evaluate(x_train, y_train)
train_var = y_train.var()

val_mse = model.evaluate(x_val, y_val)
val_var = y_val.var()

print(f'Train MSE: {train_mse}')
print(f'Train r^2: {(train_var - train_mse)/train_var}')

print(f'Validation MSE: {val_mse}')
print(f'Validation r^2: {(val_var - val_mse)/val_var}')

Still not very good.  Let's try standardizing the quantitative features now that they are better scaled.

In [ ]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()
scale_cols = ['log(revenue)', 'log(expenses)', 'log(employees)', 'log(market cap)']
s.fit(x_train[scale_cols])

x_train.loc[:, scale_cols] = s.transform(x_train[scale_cols])
x_val.loc[:, scale_cols]   = s.transform(x_val[scale_cols])
x_test.loc[:, scale_cols]  = s.transform(x_test[scale_cols])

x_train.head()

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(units = 50, activation = 'sigmoid', input_shape = (x_train.shape[1],)),
    keras.layers.Dense(units = 10, activation = 'sigmoid'),
    keras.layers.Dense(units = 1),
])

model.compile(optimizer='adam',
              loss='mse')
model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), verbose=0)

train_mse = model.evaluate(x_train, y_train)
train_var = y_train.var()

val_mse = model.evaluate(x_val, y_val)
val_var = y_val.var()

print(f'Train MSE: {train_mse}')
print(f'Train r^2: {(train_var - train_mse)/train_var}')

print(f'Validation MSE: {val_mse}')
print(f'Validation r^2: {(val_var - val_mse)/val_var}')

Still not very good, but it is better.  We don't seem to be overfitting, so what if we trained our model more?

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(units = 50, activation = 'sigmoid', input_shape = (x_train.shape[1],)),
    keras.layers.Dense(units = 10, activation = 'sigmoid'),
    keras.layers.Dense(units = 1),
])

model.compile(optimizer='adam',
              loss='mse')
model.fit(x_train, y_train, epochs=500, validation_data=(x_val, y_val), verbose=0)

train_mse = model.evaluate(x_train, y_train)
train_var = y_train.var()

val_mse = model.evaluate(x_val, y_val)
val_var = y_val.var()

print(f'Train MSE: {train_mse}')
print(f'Train r^2: {(train_var - train_mse)/train_var}')

print(f'Validation MSE: {val_mse}')
print(f'Validation r^2: {(val_var - val_mse)/val_var}')

There we go.  This is much better!  What if we train our model even longer?

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(units = 50, activation = 'sigmoid', input_shape = (x_train.shape[1],)),
    keras.layers.Dense(units = 10, activation = 'sigmoid'),
    keras.layers.Dense(units = 1),
])

model.compile(optimizer='adam',
              loss='mse')
model.fit(x_train, y_train, epochs=1000, validation_data=(x_val, y_val), verbose=0)

train_mse = model.evaluate(x_train, y_train)
train_var = y_train.var()

val_mse = model.evaluate(x_val, y_val)
val_var = y_val.var()

print(f'Train MSE: {train_mse}')
print(f'Train r^2: {(train_var - train_mse)/train_var}')

print(f'Validation MSE: {val_mse}')
print(f'Validation r^2: {(val_var - val_mse)/val_var}')

This is starting to look pretty good!  We can work on this more if we have more time, but I expect at this point we'll be done.